In [ ]:
import geopandas
import numpy
import rioxarray
import xarray as xr
import os
import cftime
import matplotlib.pyplot as plt
from datetime import timedelta
from dateutil.relativedelta import relativedelta
from geocube.api.core import make_geocube
import pandas as pd
import glob

In [ ]:
muni_data = geopandas.read_file("../clip/brazil_munis_interviews.geojson").to_crs('EPSG:4326')
muni_data['CD_MUN'] = muni_data['CD_MUN'].astype(int)

In [ ]:
if not os.path.isfile('./data/chirps-v2.0.monthly_muni.nc'):
    muni_bounds = muni_data.total_bounds
    precip_ds = xr.open_dataset('./data/chirps-v2.0.monthly.nc')
    precip_ds_muni = precip_ds.sel(latitude=slice(muni_bounds[1], muni_bounds[3]),
                longitude=slice(muni_bounds[0], muni_bounds[2]))
    precip_ds_muni.to_netcdf('./data/chirps-v2.0.monthly_muni.nc')

In [ ]:
muni_rio_ds = rioxarray.open_rasterio('./data/chirps-v2.0.monthly_muni.nc')
muni_rio_ds.rio.write_crs(4326, inplace=True)

#Not clipping, not necessary
precip_clip=muni_rio_ds
# precip_clip = muni_rio_ds.rio.clip(
#     muni_data.geometry.values, muni_data.crs, from_disk=True
# )

In [ ]:

muni_out_grid = make_geocube(
    vector_data=muni_data,
    measurements=["CD_MUN"],
    like=precip_clip, # ensure the data are on the same grid
)

In [ ]:

muni_out_grid["tp"] = (precip_clip.dims, precip_clip.values, precip_clip.attrs, precip_clip.encoding)

In [ ]:
out_grid = make_geocube(
    vector_data=muni_data,
    measurements=["CD_MUN"],
    like=precip_clip, # ensure the data are on the same grid
)

In [ ]:
out_grid["tp"] = (precip_clip.dims, precip_clip.values, precip_clip.attrs, precip_clip.encoding)

In [ ]:
grouped_tp = muni_out_grid.drop("spatial_ref").groupby(muni_out_grid.CD_MUN)

In [ ]:
tp_mean =grouped_tp.mean()

In [ ]:
precip_clip.mean(dim=['y','x']).isel(time=slice(-280,-1)).plot.line(x='time')

In [ ]:
tp_mean['time'] = precip_clip['time']
tp_mean = tp_mean.sortby('time')
tp_mean_rainy_offset = tp_mean.copy()
tp_mean_rainy_offset['time'] = (tp_mean.indexes['time'].to_datetimeindex() + pd.DateOffset(months=+8)).values

In [ ]:
# First print annual precip by muni
muni_annual_tp = tp_mean['tp'].groupby('time.year').sum('time').sel(year=slice(1984,2024)).groupby('CD_MUN').mean(dim='year')

muni_names_dict = {
    5101803: 'Barra do Garças',
    5102702: 'Canarana',
    5106257: 'Nova Xavantina',
    5107065: 'Querência',
}

for muni in muni_names_dict.keys():
    print(muni_names_dict[muni])
    print(muni_annual_tp.sel(CD_MUN=muni).values)

In [ ]:
tp_mean_rainy_offset['tp'].mean(dim='CD_MUN').groupby('time.year').sum('time').sel(year=slice(1982,2024)).plot.line(x='year')

In [ ]:
monthly_start =cftime.DatetimeProlepticGregorian(2019, 9, 1, 0, 0, 0, 0, has_year_zero=True)
monthly_end =cftime.DatetimeProlepticGregorian(2024, 9, 1, 0, 0, 0, 0, has_year_zero=True)
monthly_start_inset = 2020
monthly_end_inset = 2024
interview_time_start = cftime.DatetimeProlepticGregorian(2023, 8, 1, 0, 0, 0, 0, has_year_zero=True)
interview_time_end = cftime.DatetimeProlepticGregorian(2023, 10, 1, 0, 0, 0, 0, has_year_zero=True)


In [ ]:
ax = tp_mean['tp'].mean(dim='CD_MUN').sel(time=slice(monthly_start,monthly_end)).plot.line(x='time')
monthly_end =cftime.DatetimeProlepticGregorian(2024, 9, 1, 0, 0, 0, 0, has_year_zero=True)

# Create data for the plot
y_lower = [0, 0]  # y-values for the lower boundary of shading
y_upper = [600, 600]  # y-values for the upper boundary of shading

# Plot the shading between the two vertical lines
plt.fill_between([interview_time_start, interview_time_end], y_lower, y_upper, color='green', alpha=0.25)
plt.axvline(interview_time_start,color='darkgreen')
plt.axvline(interview_time_end,color='darkgreen')
plt.ylim(0,y_upper[0])

In [ ]:

def read_process_region_csv(csv):
    temp_df = pd.read_csv(csv)
    temp_df['satellite'] = os.path.basename(csv)[:3]
    temp_df['year'] = int(os.path.basename(csv)[4:8])

    return temp_df

muni_csvs = glob.glob('../clean_summarize/out/v3_interviews_only/*.csv')
muni_csvs.sort()
muni_list = [read_process_region_csv(csv) for csv in muni_csvs]
muni_df = pd.concat(muni_list)
# muni_df = muni_df.sort_index()
muni_df = muni_df.loc[~((muni_df.satellite=='ls7')&(muni_df.year>2019))]

In [ ]:
# Full Image
plt.rcParams.update({'font.size': 18})
fig, axs = plt.subplots(1, 2, figsize = (16, 8))

tp_mean_rainy_offset['tp'].mean(dim='CD_MUN'
                   ).groupby('time.year'
                             ).sum('time'
                                   ).sel(year=slice(1984,2024)
                                         ).plot.line(x='year', ax=axs[0])
axs[0].set_ylim(0, 2200)
# Creating inset
x1, x2, y1, y2 = monthly_start_inset, monthly_end_inset, 1000, 2000   # Subregion of the original image
axins = axs[0].inset_axes(
    [0.2, 0.12, 0.65, 0.4],
    xlim=[monthly_start, monthly_end],ylim=[0,600])
tp_mean['tp'].mean(dim='CD_MUN'
                        ).sel(time=slice(monthly_start,monthly_end)
                              ).plot.line(x='time', ax=axins, linestyle='--')
axins.axvline(interview_time_start,linestyle='--',color='darkgreen')
axins.axvline(interview_time_end,linestyle='--',color='darkgreen')
axins.fill_between([interview_time_start, interview_time_end], y_lower, y_upper, color='green', alpha=0.25)
axs[0].indicate_inset((x1, y1, x2-x1, y2-y1), axins)
axs[0].set_xlabel('Year')
axs[0].set_ylabel('Annual Precipitation (mm)')
axins.set_ylabel('Monthly Precip (mm)', size=14, style='italic')
axins.set_xlabel('Month', size=14, style='italic')
axins.set_title('')
axs[0].set_title('')


# Reservoir area
grouped_df = muni_df.groupby(['satellite','year']).sum()[['sum']]
for sat in ['ls5','ls7','ls8']:
    grouped_df.loc[sat].plot(ax=axs[1])
axs[1].legend(['Landsat 5','Landsat 7','Landsat 8'])

axs[1].set_xlabel('Year')
axs[1].set_ylabel('Total Reservoir Area (ha)')

In [ ]:

# Full Image
plt.rcParams.update({'font.size': 12})
axes_width_ratios=[1, 0.5]
fig, axs = plt.subplots(1, 2, figsize = (7.5, 4.5),
                       gridspec_kw={"width_ratios":axes_width_ratios})

# tp_mean_rainy_offset['tp'].mean(dim='CD_MUN'
tp_mean['tp'].mean(dim='CD_MUN'
                   ).groupby('time.year'
                             ).sum('time'
                                   ).sel(year=slice(1984,2024)
                                         ).plot.line(x='year', ax=axs[0], label='Annual precip.', marker='+')
axs[0].set_ylim(0, 2300)
# Creating inset
x1, x2, y1, y2 = monthly_start_inset, monthly_end_inset, 1000, 2000   # Subregion of the original image
axins = axs[0].inset_axes(
    [0.2, 0.14, 0.65, 0.4],
    xlim=[monthly_start, monthly_end],ylim=[0,600])
tp_mean['tp'].mean(dim='CD_MUN'
                        ).sel(time=slice(monthly_start,monthly_end)
                              ).plot.line(x='time', ax=axins, linestyle='--', label='Monthly precip.')
axins.axvline(interview_time_start,linestyle='--',color='darkgreen')
axins.axvline(interview_time_end,linestyle='--',color='darkgreen')
axins.fill_between([interview_time_start, interview_time_end], y_lower, y_upper, color='green', alpha=0.25, label='Interview period')
axs[0].indicate_inset((x1, y1, x2-x1, y2-y1), axins)
axs[0].set_xlabel('Year')
axs[0].set_ylabel('Annual Precipitation (mm)')
axins.set_ylabel('Monthly Precip (mm)', size=14, style='italic')
axins.set_xlabel('Month', size=14, style='italic')
axins.set_title('')
axs[0].set_title('')


# Reservoir area
handles, labels = axs[0].get_legend_handles_labels()
handles2, labels2 = axins.get_legend_handles_labels()
handles = handles + handles2
labels = labels + labels2

axs[1].set_axis_off()
axs[1].legend(handles, labels, loc=10)
fig.tight_layout()
plt.savefig('/home/ksolvik/research/reservoirs/figs/ch3/precip.jpg', dpi=300,
            pil_kwargs={'quality':95},
            bbox_inches='tight')